In [ ]:
import pandas as pd
import yfinance as yf
import yahoofinancials

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

tf.compat.v1.enable_v2_behavior()

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
#import pyvirtualdisplay

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

In [ ]:
tsla_test = yf.download('TSLA', 
                      start='2020-06-01', 
                      end='2020-06-02',
                     interval = '1m')
#tsla_test['Close'].plot()
print(tsla_test.shape)
print(tsla_test.index[tsla_test.shape[0]-1])
tsla_mod = tsla_test.drop(tsla_test.index[tsla_test.shape[0]-1])
print(tsla_mod.shape)
#help(tsla_test)
#help(yf.ticker)
print(tsla_mod)
tsla_mod['Close'].plot()


In [ ]:
from tf_agents.environments import py_environment

# ticker  = yf.Ticker('TSLA')
# tsla_df = ticker.history(period="max")
#'data = tsla_mod['Close']

class StockAttackEnv(py_environment.PyEnvironment):

  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=2, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(1,), dtype=np.int32, minimum=0, name='observation')
    self._episode_ended = False
    self.episodeTimeStepNumber = 0
    self.initCapital           = 1000
    self._state                 = self.initCapital
    self.ownedStock            = 0
    self.data = tsla_mod['Close']
    self.cost = 1

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._state = self.initCapital #start with value of 100
    self._episode_ended = False
    self.episodeTimeStepNumber = 0
    return ts.restart(np.array([self._state], dtype=np.int32))

  def _step(self, action):
    
    if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
      return self.reset()

    #define observation = current stock price, ATTENTION obseration could be known values?
    self._observation_spec = self.data[self.episodeTimeStepNumber]
    #self._observation_spec = self.episodeTimeStepNumber
    
    # Make sure episodes don't go on forever.
    price_buy  = self.data[self.episodeTimeStepNumber]+self.cost
    price_sell = self.data[self.episodeTimeStepNumber]-self.cost
    if action == 1:
    #start out with buying/selling always the max amount of stocks
        howManyStocksCanIAfford = (self._state-self._state%price_buy)/price_buy
        if howManyStocksCanIAfford > 0:
            self._state = self._state - (price_buy - self.cost) *howManyStocksCanIAfford
            self.ownedStock = self.ownedStock + howManyStocksCanIAfford
            
            
    elif action == 0:
      #do nothing
        pass
    elif action == 2:
        self._state     = self._state + price_sell * self.ownedStock
        self.ownedStock = 0
    else:
      raise ValueError('`action` should be 0 or 1.')

    reward = self._state + price_sell * self.ownedStock - self.initCapital
    if self.data.shape[0]-1>self.episodeTimeStepNumber:
        self.episodeTimeStepNumber +=1
        #import pdb; pdb.set_trace()
        return ts.transition(np.array([self._state], dtype=np.int32), reward, discount=0.0)
    else:
      self._episode_ended = True
      return ts.termination(np.array([self._state], dtype=np.int32), reward)

    #print(action)
    
    

In [ ]:
# this does not work, no clue why
environment = StockAttackEnv()
utils.validate_py_environment(environment, episodes=5)


In [ ]:
#not sure if reward has to cumulative, if yes make reward the difference to the init state
#e.g. init state = 1000, current state = 1100  reward = +100

buy_action = np.array(1, dtype=np.int32)
sell_action = np.array(2, dtype=np.int32)
wait_action = np.array(0, dtype=np.int32)

environment = StockAttackEnv()
time_step = environment.reset()
#print(time_step)
cumulative_reward = time_step.reward
current_observation = environment.observation_spec()
print(cumulative_reward, current_observation)

time_step = environment.step(buy_action)
#print(time_step)
cumulative_reward += time_step.reward
current_observation = environment.observation_spec()
print(cumulative_reward, current_observation)

time_step = environment.step(wait_action)
cumulative_reward += time_step.reward
current_observation = environment.observation_spec()
print(cumulative_reward, current_observation)

time_step = environment.step(sell_action)
#print(time_step)
cumulative_reward += time_step.reward
current_observation = environment.observation_spec()
print(cumulative_reward, current_observation)

print('Final Reward = ', cumulative_reward)


In [ ]:
#train on simple environment

#parameters
num_iterations = 20 # @param {type:"integer"}

initial_collect_steps = 20  # @param {type:"integer"} 
collect_steps_per_iteration = 100  # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 16  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 2  # @param {type:"integer"}

num_eval_episodes = 1  # @param {type:"integer"}
eval_interval = 1  # @param {type:"integer"}

PyEnv = StockAttackEnv()
env = tf_py_environment.TFPyEnvironment(PyEnv)
env.reset()
action_spec = PyEnv.action_spec()
#print(action_spec.shape)

fc_layer_params = (1000,)

q_net = q_network.QNetwork(
    env.observation_spec(),
    env.action_spec(),
    fc_layer_params=fc_layer_params)

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

step_counter = tf.compat.v2.Variable(0)

agent = dqn_agent.DqnAgent(
    env.time_step_spec(),
    env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=step_counter)

agent.initialize()

print(agent.collect_data_spec)

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=env.batch_size,
    max_length=replay_buffer_max_length)

def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]

In [ ]:
def collect_episode(environment, policy, num_episodes):

  episode_counter = 0
  environment.reset()

  while episode_counter < num_episodes:
    time_step = environment.current_time_step()
    action_step = policy.action(time_step)
    next_time_step = environment.step(action_step.action)
    traj = trajectory.from_transition(time_step, action_step, next_time_step)

    # Add trajectory to the replay buffer
    replay_buffer.add_batch(traj)

    if traj.is_boundary():
      episode_counter += 1



    
agent.train = common.function(agent.train)

# Reset the train step
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(env, agent.policy, num_eval_episodes)
returns = [avg_return]
print(avg_return)

for _ in range(num_iterations):

  # Collect a few episodes using collect_policy and save to the replay buffer.
  collect_episode(
      env, agent.collect_policy, collect_steps_per_iteration)

  dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, 
    sample_batch_size=batch_size, 
    num_steps=2).prefetch(3)
  iterator = iter(dataset)
  iterator.next()
  # Use data from the buffer and update the agent's network.
  #experience = replay_buffer.gather_all()
  experience, unused_info = next(iterator)
  #import pdb; pdb.set_trace()
  train_loss = agent.train(experience).loss
  replay_buffer.clear()

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)


In [ ]:
steps = range(0, num_iterations + 1, eval_interval)
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
#plt.ylim(top=250)